In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


请使用GPU运行此notebook

# Huggingface Llama2 + LoRA (Recommended)

需要request权限，还在审核之中

In [ ]:
!pip install -q huggingface_hub
!pip install -q -U trl transformers accelerate peft
!pip install -q -U datasets bitsandbytes einops wandb

# Uncomment to install new features that support latest models like Llama 2
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## demo finetuning code
这个自带wandb


more info:
https://huggingface.co/learn/audio-course/en/chapter4/fine-tuning#fine-tuning-the-model

TODO:
1. add metrics
2. hyperparameter search

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

dataset_name = "TiffanyCheng/LLM_Bias_EECS182_Project" # e.g. https://huggingface.co/datasets/YeungNLP/moss-003-sft-data
dataset = load_dataset(dataset_name, split="train")

shuffled_dataset = dataset.shuffle(seed=42) # Split the dataset into train+validation and test
train_valid_split = shuffled_dataset['train'].train_test_split(test_size=0.2)
train_valid_dataset = train_valid_split['train']
test_dataset = train_valid_split['test']

base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1

################## Hyper-Parameter #########################
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

################## Hyper-Parameter ##########################

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2, # if out of memory, reduce this
    gradient_accumulation_steps=6, # then add this
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500,
    push_to_hub = True,
    num_train_epochs=10 # may use early stop
)


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Map:   0%|          | 0/1074551 [00:00<?, ? examples/s]

KeyError: ignored

In [ ]:
max_seq_length = 512 # if oom, may reduce this!

# TODO: check whether the sequence is completed after cutting

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['conversation'])):
        text = f"### Question: {example['conversation'][i][0]['human']}\n ### Answer: {example['conversation'][i][0]['assistant']}"
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)

trainer.train()

import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

Hyper search

In [ ]:
# def wandb_hp_space(trial):
#     return {
#         "method": "random",
#         "metric": {"name": "objective", "goal": "minimize"},
#         "parameters": {
#             "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
#             "per_device_train_batch_size": {"values": [16, 32]}
#         },
#     }


In [ ]:
# best_trial = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="wandb",
#     hp_space=wandb_hp_space,
#     n_trials=20,
# #     compute_objective=compute_objective,
# )
